In [ ]:
# Step 1: Install required libraries and Tesseract OCR
!pip install pytesseract
!pip install pillow
!pip install gtts
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev

# Step 2: Import necessary libraries
from PIL import Image
import pytesseract
from google.colab import files
from gtts import gTTS
from IPython.display import Audio, display

# Step 3: Configure pytesseract to use the installed Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# Step 4: Upload an image
print("Please upload an image file:")
uploaded = files.upload()

# Step 5: Load the image using PIL
for filename in uploaded.keys():
    image = Image.open(filename)
    break

# Step 6: Use pytesseract to extract text from the image
text = pytesseract.image_to_string(image)

# Step 7: Print the extracted text
print("Extracted Text:")
print(text)

# Step 8: Convert the extracted text to speech using gTTS
tts = gTTS(text)
tts.save("story.mp3")

# Step 9: Play the audio
display(Audio("story.mp3", autoplay=True))


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.
Reading pac

Saving story2.png to story2 (1).png
Extracted Text:
 

Tue Repwoop TREE

My father once told me a story about an old redwood
tree—how she stood tall and proud—her sprawling limbs
clothed in emerald green. With a smile, he described her as
a mere sapling, sheltered by her elders and basking in the
safety of the warm, dappled light. But as this tree grew
taller, she found herselfat the mercy of the cruel wind and
the vicious rain. Together, they tore relentlessly at her
pretty boughs, until she felt as though her heart would
split in two.

After a long, thoughtful pause, my father turned to me and
said, “My daughter, one day the same thing will happen to
you. And when that time comes, remember the redwood
tree. Do not worry about the cruel wind or the vicious
rain—but do as that tree did and just keep growing.”



In [ ]:
# Step 1: Install required libraries and Tesseract OCR
!pip install telebot
!pip install pytesseract
!pip install pillow
!pip install gtts

# Install Tesseract OCR
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!apt-get install -y espeak

# Step 2: Import necessary libraries
import telebot
from PIL import Image
import pytesseract
from gtts import gTTS
from io import BytesIO

# Step 3: Configure pytesseract to use the installed Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# Step 4: Initialize the bot and define message handling functions
bot_token = '7204979787:AAFMVSL_8ndRUf5AOGwSZ7GimKGrsC1xPfE'
bot = telebot.TeleBot(bot_token)

# Dictionary to store user states
user_state = {}

@bot.message_handler(commands=['start'])
def handle_start(message):
    bot.send_message(message.chat.id, 'Hi! Send me an image and I will read the text for you.')

@bot.message_handler(commands=['help'])
def handle_help(message):
    bot.send_message(message.chat.id, 'Send me an image and I will extract the text and read it out to you.')

@bot.message_handler(content_types=['text'])
def handle_text(message):
    if message.text.lower() == '/yes':
        bot.send_message(message.chat.id, 'Give me an image of the story and I will recite it for you.')
        user_state[message.chat.id] = 'waiting_for_image'
    elif message.text.lower() == '/no':
        bot.send_message(message.chat.id, 'Good night. Have a good sleep!')
    else:
        bot.send_message(message.chat.id, "I can only process images. Please send an image.")

@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    # Check if message contains photo
    if message.photo:
        photo_id = message.photo[-1].file_id
        file_info = bot.get_file(photo_id)
        downloaded_file = bot.download_file(file_info.file_path)

        # Process image with OCR
        text = extract_text_from_image(downloaded_file)

        # Convert text to speech
        audio_file = convert_text_to_speech(text)

        # Send audio file to user
        bot.send_voice(message.chat.id, audio_file)

        # Ask if user wants to hear another story
        ask_another_story(message)
    else:
        bot.send_message(message.chat.id, 'Please upload an image.')

def ask_another_story(message):
    bot.send_message(message.chat.id, 'Do you want to hear another story? Send /yes or /no.')
    user_state[message.chat.id] = 'waiting_for_response'

@bot.message_handler(func=lambda message: user_state.get(message.chat.id) == 'waiting_for_response', content_types=['text'])
def handle_another_story_response(message):
    response = message.text.strip().lower()
    if response == '/yes':
        bot.send_message(message.chat.id, 'Give me another image of the story.')
        user_state[message.chat.id] = 'waiting_for_image'
    elif response == '/no':
        bot.send_message(message.chat.id, 'Good night. Have a good sleep!')
        user_state[message.chat.id] = None
    else:
        bot.send_message(message.chat.id, 'Please send /yes or /no.')

@bot.message_handler(func=lambda message: user_state.get(message.chat.id) == 'waiting_for_image', content_types=['photo'])
def handle_new_image(message):
    photo_id = message.photo[-1].file_id
    file_info = bot.get_file(photo_id)
    downloaded_file = bot.download_file(file_info.file_path)

    # Process image with OCR
    text = extract_text_from_image(downloaded_file)

    # Convert text to speech
    audio_file = convert_text_to_speech(text)

    # Send audio file to user
    bot.send_voice(message.chat.id, audio_file)

    # Ask if user wants to hear another story
    ask_another_story(message)

def extract_text_from_image(image_file):
    image = Image.open(BytesIO(image_file))
    text = pytesseract.image_to_string(image)
    return text.strip()

def convert_text_to_speech(text):
    tts = gTTS(text)
    audio_stream = BytesIO()
    tts.write_to_fp(audio_stream)
    audio_stream.seek(0)
    return audio_stream

# Step 5: Start the bot
bot.polling()


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.
Reading pac